# Convolutional AutoEncoder (CAE) Tensorflow
Train a variational autoencoder with MNIST dataset

#### References:
* http://kvfrans.com/variational-autoencoders-explained/
* https://github.com/kvfrans/variational-autoencoder
* https://github.com/int8/VAE_tensorflow
* http://int8.io/variational-autoencoder-in-tensorflow/
* http://blog.fastforwardlabs.com/2016/08/22/under-the-hood-of-the-variational-autoencoder-in.html
* http://blog.fastforwardlabs.com/2016/08/12/introducing-variational-autoencoders-in-prose-and.html
* https://blog.keras.io/building-autoencoders-in-keras.html
* https://jaan.io/what-is-variational-autoencoder-vae-tutorial/
* https://arxiv.org/pdf/1606.05908.pdf
* https://arxiv.org/pdf/1312.6114.pdf
* http://wiseodd.github.io/techblog/2016/12/10/variational-autoencoder/
* https://www.tensorflow.org/get_started/embedding_viz
* https://www.youtube.com/watch?v=eBbEDRsCmv4
* https://github.com/normanheckscher/mnist-tensorboard-embeddings
* http://projector.tensorflow.org/
* https://www.youtube.com/watch?v=P78QYjWh5sM
* https://www.youtube.com/watch?v=bbOFvxbMIV0

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)

SAVE_FOLDER='/tmp/vae_cnn'

# Delete directory if exist
if os.path.exists(SAVE_FOLDER):    
    os.system("rm -rf " + SAVE_FOLDER)

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

/Users/leonardo_a/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [2]:
from models import CAE_CNN
model = CAE_CNN(latent_size = 20)
model_in = model.input
model_out = model.output
model_out_flat = model.output_flat

### Define loss

In [3]:
with tf.name_scope("CAE_LOSS"):
    # Binary cross entropy
    bce = -tf.reduce_sum(model_in * tf.log(1e-8 + model_out_flat) + (1-model_in) * tf.log(1e-8 + 1 - model_out_flat),1)
    loss = tf.reduce_mean(bce)
    
    # L2 Loss
    #loss = tf.nn.l2_loss(model_in-model_out_flat)            

# Solver configuration
with tf.name_scope("Solver"):
    train_step = tf.train.AdamOptimizer(0.00001).minimize(loss)

### Build Graph

In [4]:
init = tf.global_variables_initializer()

# Avoid allocating the whole memory
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.200)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

#sess = tf.Session()
sess.run(init)

### Add some tensors to observe on Tensorboard

In [5]:
tf.summary.image("input_image", model.image_in, 4)
tf.summary.image("output_image", model_out, 4)
tf.summary.scalar("loss", loss)

merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter(SAVE_FOLDER)
writer.add_graph(sess.graph)

# Create saver object
saver = tf.train.Saver()

### Train

In [ ]:
num_epoch = 200
batch_size = 10
for epoch in range(num_epoch):
    for i in range(int(mnist.train.num_examples / batch_size)):
        # Get batch of 50 images
        batch = mnist.train.next_batch(batch_size)

        # Dump summary
        if i % 5000 == 0:            
            # Other summaries
            s = sess.run(merged_summary, feed_dict={model_in:batch[0]})
            writer.add_summary(s,i)                     


        # Train actually here (Also get loss value)    
        _, val_loss = sess.run((train_step, loss), feed_dict={model_in:batch[0]})

        #if i % 5000:
            #print('Generation loss: %d, latent loss: %d' % (np.mean(gen_loss), np.mean(lat_loss)))
    print('Epoch: %d/%d loss:%d' % (epoch, num_epoch, val_loss))
    print('Save model:', epoch)
    saver.save(sess, os.path.join(SAVE_FOLDER, "model.ckpt"), epoch)

Epoch: 0/200 loss:219
Save model: 0
Epoch: 1/200 loss:165
Save model: 1
Epoch: 2/200 loss:150
Save model: 2
Epoch: 3/200 loss:115
Save model: 3
Epoch: 4/200 loss:105
Save model: 4
Epoch: 5/200 loss:91
Save model: 5
Epoch: 6/200 loss:94
Save model: 6
Epoch: 7/200 loss:96
Save model: 7
Epoch: 8/200 loss:76
Save model: 8
Epoch: 9/200 loss:83
Save model: 9
Epoch: 10/200 loss:91
Save model: 10
Epoch: 11/200 loss:73
Save model: 11
Epoch: 12/200 loss:73
Save model: 12
Epoch: 13/200 loss:79
Save model: 13
Epoch: 14/200 loss:78
Save model: 14
Epoch: 15/200 loss:82
Save model: 15
Epoch: 16/200 loss:77
Save model: 16
Epoch: 17/200 loss:74
Save model: 17
Epoch: 18/200 loss:87
Save model: 18
Epoch: 19/200 loss:97
Save model: 19
Epoch: 20/200 loss:70
Save model: 20
Epoch: 21/200 loss:76
Save model: 21
Epoch: 22/200 loss:76
Save model: 22
Epoch: 23/200 loss:91
Save model: 23
Epoch: 24/200 loss:77
Save model: 24
Epoch: 25/200 loss:79
Save model: 25
Epoch: 26/200 loss:70
Save model: 26
Epoch: 27/200 lo